In [4]:
import numpy as np
%matplotlib qt
%load_ext autoreload
%autoreload 2
import mne
from mne.datasets.brainstorm import bst_raw
from mne.io import read_raw_ctf
import sys; sys.path.insert(0, '../../esinet')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model
import os
sys.path.insert(0, "../")
from laura import *

print(__doc__)

tmin, tmax, event_id = -0.1, 0.3, 2  # take right-hand somato
reject = dict(mag=4e-12, eog=250e-6)

data_path = bst_raw.data_path()

raw_path = os.path.join(data_path, 'MEG', 'bst_raw', 
    'subj001_somatosensory_20111109_01_AUX-f.ds')
# Here we crop to half the length to save memory
raw = read_raw_ctf(raw_path).crop(0, 180).load_data()
raw.plot()

# set EOG channel
raw.set_channel_types({'EEG058': 'eog'})
raw.set_eeg_reference('average', projection=True)

# show power line interference and remove it
raw.plot_psd(tmax=60., average=False)
raw.notch_filter(np.arange(60, 181, 60), fir_design='firwin')

events = mne.find_events(raw, stim_channel='UPPT001')

# pick MEG channels
picks = mne.pick_types(raw.info, meg=True, eeg=False, stim=False, eog=True,
                       exclude='bads')

# Compute epochs
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=(None, 0), reject=reject, preload=False)

# compute evoked
evoked = epochs.average()

# remove physiological artifacts (eyeblinks, heartbeats) using SSP on baseline
evoked.add_proj(mne.compute_proj_evoked(evoked.copy().crop(tmax=0)))
evoked.apply_proj()

# fix stim artifact
mne.preprocessing.fix_stim_artifact(evoked)

# correct delays due to hardware (stim artifact is at 4 ms)
evoked.shift_time(-0.004)

# plot the result
evoked.plot(time_unit='s')

# show topomaps
evoked.plot_topomap(times=np.array([0.016, 0.030, 0.060, 0.070]),
                    time_unit='s', title="True Evoked")

noise_cov = mne.compute_covariance(epochs, tmax=0)
fwd_small = create_forward_model(info=evoked.info, sampling='ico3')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Automatically created module for IPython interactive environment
ds directory : C:\Users\Lukas\mne_data\MNE-brainstorm-data\bst_raw\MEG\bst_raw\subj001_somatosensory_20111109_01_AUX-f.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       0.84   69.49    0.00 mm <->    0.84   69.49   -0.00 mm (orig :  -44.30   51.45 -252.43 mm) diff =    0.000 mm
      -0.84  -69.49    0.00 mm <->   -0.84  -69.49    0.00 mm (orig :   46.28  -53.58 -243.47 mm) diff =    0.000 mm
      86.41    0.00    0.00 mm <->   86.41    0.00    0.00 mm (orig :   63.60   55.82 -230.26 mm) diff =    0.000 mm
    Coordinate transformations established.
    Reading digitizer points from ['C:\\Users\\Lukas\\mne_data\\MNE-brainstorm-data\\bst_raw\\MEG\\bst_raw\\subj001_somatosensory_20111109_01_AUX-f.ds\\subj00111092011.pos']...
    Polhemus da

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.2s remaining:    1.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(

# Calc and plot LAURA

In [7]:
subjects_dir = mne.datasets.sample.data_path() + '\\subjects'
clim=dict(kind='percent', pos_lims=(0,50,99))
plot_params = dict(surface='white', hemi='both', verbose=0, subjects_dir=subjects_dir, clim=clim)

stc_laura = apply_laura(epochs.average(), fwd_small, reg=0.0005, noise_cov=None)
stc_laura.plot(**plot_params)

# evoked_laura = util.get_eeg_from_source(stc_laura, fwd_small, evoked.info, tmin=evoked.tmin)
# evoked.plot_topomap(times=np.array([0.016, 0.030, 0.060, 0.070]),
#                     time_unit='s', title="True Evoked")
# evoked_laura.plot_topomap(times=np.array([0.016, 0.030, 0.060, 0.070]),
#                     time_unit='s', title='LAURA')

Removing projector <Projection | Average EEG reference, active : True, n_channels : 1>
Removing 5 compensators from info because not all compensation channels were picked.
-- number of adjacent vertices : 1284


c:\Users\Lukas\Documents\projects\laura\dev\..\laura\laura.py:129: RuntimeWarning: divide by zero encountered in power
  else:


Using control points [6.23124876e-13 7.81734361e-13 1.58704275e-12]


  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Channels marked as bad: none


# elor

In [56]:
method = "eLORETA"
snr = 3.
lambda2 = 1. / snr ** 2
# noise_cov = mne.compute_covariance(
#     epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)

inverse_operator = mne.minimum_norm.make_inverse_operator(
    evoked.info, fwd_small, noise_cov, loose='auto', depth=None, fixed=True, 
    verbose=False)
    
stc_elor, residual = mne.minimum_norm.apply_inverse(epochs.average(), inverse_operator, lambda2,
                              method=method, return_residual=True, verbose=False)
brain = np.abs(stc_elor).plot(**plot_params)
brain.add_text(0.1, 0.9, 'eLORETA on auditory data', 'title',
               font_size=14)
# Plot predicted EEG
epochs.load_data()
evoked_elor = util.get_eeg_from_source(stc_elor, fwd_small, evoked.info, tmin=0.)
evoked_elor.plot()
evoked_elor.plot_joint(title='eLORETA')


Removing projector <Projection | Average EEG reference, active : True, n_channels : 1>


KeyboardInterrupt: 

In [77]:
from esinet import Simulation
settings=dict(number_of_sources=2, extents=(25, 35), duration_of_trial=0.1)
sim = Simulation(fwd_small, evoked.info, settings=settings).simulate(2)


Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 1002.22it/s]


source data shape:  (1284, 120) (1284, 120)


100%|██████████| 2/2 [00:00<00:00, 19.28it/s]


In [79]:
sim.source_data[0].plot(**plot_params)
evoked_sim = sim.eeg_data[0].average()
stc_laura = apply_laura_2(evoked_sim, fwd_small)
stc_laura.plot(**plot_params)

prepare gain
calc laura


c:\Users\Lukas\Documents\projects\laura\dev\..\laura\laura.py:222: RuntimeWarning: divide by zero encountered in power
  W = np.identity(A.shape[0])


calc W_j
calc W_d
calc noise_term
calc G
calc y_hat
make stc


Using control points [6.83644300e-18 7.36656257e-18 9.08102094e-17]
